<a href="https://colab.research.google.com/github/zhuhadar/ai-summer-2025/blob/main/week4_day1_crewai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crew.ai Exploration
> Let's try a basic agent!

Let's try a naive approach using existing tools.

We need to do a few pip installs to get the environment setup.

```bash
python3 -m venv crewai-venv
source crewai-venv/bin/activate
pip install ipykernel "crewai[tools]" python-dotenv langchain-community pymupdf pypdf2
```

In [ ]:
!pip install ipykernel "crewai[tools]" python-dotenv langchain-community pymupdf pypdf2

In [ ]:
!pip install "crewai[tools]" langchain-community pymupdf pypdf2

In [ ]:
#simplify with google colab
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# Import the necessary libraries
from crewai import Agent, Crew, Task, Process
from crewai_tools import WebsiteSearchTool

In [ ]:
#import langchain
#help(langchain.tools)  # See if WebsiteSearchTool is listed


In [ ]:
class WebsiteSearchTool:
    def __init__(self):
        print("Tool ready")

website_search_tool = WebsiteSearchTool()


Tool ready


In [ ]:
# lily commented - from langchain.tools import WebsiteSearchTool  # Adjust to actual module if needed


## Creating our first Crew AI Crew

Let's make a Crew whose purpose is to extract text from an image.

## Instantiate the tools
We'll use the WebsiteSearchTool.

In [ ]:
# Create the VisionTool
website_search_tool = WebsiteSearchTool()

Tool ready


## Define the Agent
Visit https://docs.crewai.com/concepts/agents#direct-code-definition to learn more. Below are the full options for the agent.
This basically resolves to determining the role of the agent, specifically:
* The role title
* The goal of the agent
* The backstory of the agent



In [ ]:
agent_role = ""
agent_goal = ""
agent_backstory = ""

In [ ]:
from langchain.tools import BaseTool


In [ ]:
# lily rrmoved her own code - from crewai_tools import tool
from crewai_tools.tools import WebsiteSearchTool


In [ ]:
from crewai_tools.tools import WebsiteSearchTool

# Instantiate the tool
website_search_tool = WebsiteSearchTool()


/usr/local/lib/python3.11/dist-packages/alembic/config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [ ]:
# Create an agent with all available parameters
archivist_agent = Agent(
    role=agent_role,
    goal=agent_goal,
    backstory=agent_backstory,
    llm="gpt-4o-mini",  # Default: OPENAI_MODEL_NAME or "gpt-4"
    function_calling_llm=None,  # Optional: Separate LLM for tool calling
    memory=True,  # Default: True
    verbose=False,  # *******Changed from True to False
    allow_delegation=False,  # Default: False
    max_iter=20,  # Default: 20 iterations
    max_rpm=None,  # Optional: Rate limit for API calls
    max_execution_time=None,  # Optional: Maximum execution time in seconds
    max_retry_limit=2,  # Default: 2 retries on error
    allow_code_execution=False,  # Default: False
    code_execution_mode="safe",  # Default: "safe" (options: "safe", "unsafe")
    respect_context_window=True,  # Default: True
    use_system_prompt=True,  # Default: True
    tools=[website_search_tool],  # *******Optional: List of tools
    knowledge_sources=None,  # Optional: List of knowledge sources
    embedder=None,  # Optional: Custom embedder configuration
    system_template=None,  # Optional: Custom system prompt template
    prompt_template=None,  # Optional: Custom prompt template
    response_template=None,  # Optional: Custom response template
    step_callback=None,  # Optional: Callback function for monitoring
)

In [ ]:
# Create a task
main_task = Task(
    description="Summarize the information at the following website URL: {provided_link}",
    expected_output="A paragraph summarizing the most important points of the provided website",
    agent=archivist_agent,
)

In [ ]:
# Create a crew
crew = Crew(
    agents=[archivist_agent],
    tasks=[main_task], # <--- Used here
    process = Process.sequential,
    verbose=False # *******Changed from True to False
)

# Define the website link
website_link = "https://www.vanderbilt.edu/undergrad-datascience/minor/"

# Run the crew once
result = crew.kickoff({"provided_link": website_link})

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


In [ ]:
# Import the necessary libraries
from crewai import Agent, Crew, Task, Process
from crewai_tools import WebsiteSearchTool
from google.colab import userdata
import os
import sys

# Simplify with google colab
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Install necessary packages if not already present
!pip install ipykernel "crewai[tools]" python-dotenv langchain-community pymupdf pypdf2 -q
!pip install tqdm chromadb crewai crewai-tools -q

# Instantiate the WebsiteSearchTool
# Ensure we are using the tool from crewai_tools
website_search_tool = WebsiteSearchTool()

# Define Agent parameters
agent_role = "Website Archivist"
agent_goal = "Extract key information from a given website URL and summarize it."
agent_backstory = "You are an expert at browsing websites and distilling complex information into concise summaries."

# Create an agent
archivist_agent = Agent(
    role=agent_role,
    goal=agent_goal,
    backstory=agent_backstory,
    llm="gpt-4o-mini",
    memory=True,
    verbose=False,  # Keep verbose off to avoid RecursionError
    allow_delegation=False,
    tools=[website_search_tool],
)

# Create a task - DEFINE main_task HERE
main_task = Task(
    description="Summarize the information at the following website URL: {provided_link}",
    expected_output="A paragraph summarizing the most important points of the provided website",
    agent=archivist_agent,
)

# Create a crew - USE main_task AFTER IT'S DEFINED
crew = Crew(
    agents=[archivist_agent],
    tasks=[main_task],
    process=Process.sequential,
    verbose=False  # Keep verbose off
)

# Define the website link
website_link = "https://www.vanderbilt.edu/undergrad-datascience/minor/"

# Run the crew
result = crew.kickoff({"provided_link": website_link})


/usr/local/lib/python3.11/dist-packages/alembic/config.py:564: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


## Print the result

In [ ]:
# Print the result
print(result)

The Data Science Minor at Vanderbilt University includes a comprehensive curriculum designed to equip students with essential skills in data science. The program requires courses in Fundamentals of Data Science (DS 3100), Machine Learning, Programming, and Statistics, alongside elective courses to tailor the educational experience to students' interests. Elective options encompass various disciplines, including machine learning, statistical methods, and computational modeling. Minors must adhere to academic regulations specific to their home school, and they are encouraged to explore research opportunities in data science. For more details, students can contact the department at undergraduate.datascience@vanderbilt.edu.
